In [1]:
# imports
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
# read all words
words = open('../hafta12/assets/names.txt','r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
special_char = '.'
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi[special_char] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
block_size = 3 # context length

X,Y = [], []
for w in words[:5]:
    
    print(w)
    context = [0]*block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->',itos[ix])
        context = context[1:] + [ix] # crop and append
        
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [7]:
output_dim = 2 #acceptable since we only have 27 chars in our model
vocab_size = 27
C = torch.randn((vocab_size,output_dim))


In [8]:
stoi['e']

5

In [9]:
C[stoi['e']]

tensor([-0.0610,  1.2185])

In [10]:
# One-hot
# remember casting to float
index = F.one_hot(torch.tensor(5),num_classes=vocab_size).float()

In [11]:
#identical 
index @ C 

tensor([-0.0610,  1.2185])

In [12]:
C[5]

tensor([-0.0610,  1.2185])

In [13]:
C[5]

tensor([-0.0610,  1.2185])

In [14]:
X.shape

torch.Size([32, 3])

In [15]:
emb = C[X]

In [16]:
emb[0,2]

tensor([-0.0239, -0.3540])

In [17]:
emb.shape

torch.Size([32, 3, 2])

In [18]:
emb[10,1]

tensor([-0.5693,  0.2761])

In [19]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [20]:
emb[0,0],emb[0,1],emb[0,2]

(tensor([-0.0239, -0.3540]),
 tensor([-0.0239, -0.3540]),
 tensor([-0.0239, -0.3540]))

In [21]:
torch.cat([emb[:,0,:],emb[:,1,:],emb[:,2,:]],dim=1).shape

torch.Size([32, 6])

In [36]:
torch.unbind(torch.tensor([[1,2],[2,3]]))

(tensor([1, 2]), tensor([2, 3]))

In [37]:
len(torch.unbind(emb,dim=1))

3

In [22]:
emb_ = torch.unbind(emb,dim=1)
con_emb_ = torch.cat(emb_,dim=1)

In [23]:
con_emb_.shape

torch.Size([32, 6])

In [24]:
a = torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [27]:
a.view(9,2)

tensor([[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17]])

In [28]:
a.view(2,3,1,3)

tensor([[[[ 0,  1,  2]],

         [[ 3,  4,  5]],

         [[ 6,  7,  8]]],


        [[[ 9, 10, 11]],

         [[12, 13, 14]],

         [[15, 16, 17]]]])

In [31]:
a.storage()

/var/folders/t_/wsj99tvd57j356g69px3zny00000gn/T/ipykernel_13834/214256462.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage()


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14
 15
 16
 17
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 18]

In [35]:
h = emb.view(emb.shape[0],6) @ W1 + b1

In [36]:
h.shape

torch.Size([32, 100])

In [37]:
h = torch.tanh(h)

In [38]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [39]:
logits = h@ W2 +b2

In [40]:
logits.shape

torch.Size([32, 27])

In [41]:
counts = logits.exp()
prob = counts / counts.sum(1,keepdims=True)

In [44]:
sum(prob[0])

tensor(1.0000)

In [47]:
prob[torch.arange(32),Y]

tensor([1.3487e-07, 9.9459e-01, 1.2242e-10, 4.1110e-06, 3.4077e-02, 1.7999e-07,
        2.6193e-07, 1.9907e-13, 8.5234e-08, 9.1077e-14, 2.6085e-03, 1.3922e-06,
        5.4480e-05, 4.9832e-06, 1.2914e-08, 4.5704e-05, 1.3739e-11, 3.2650e-07,
        1.8317e-06, 1.7511e-07, 7.3633e-09, 3.8155e-08, 3.3649e-10, 1.7545e-01,
        1.5464e-05, 5.9514e-05, 6.1656e-06, 5.2414e-09, 9.9202e-08, 5.4649e-07,
        1.3019e-03, 1.1142e-03])

In [48]:
loss = -prob[torch.arange(32),Y].log().mean()
loss

tensor(14.1943)

In [77]:
block_size = 3 # context length

X,Y = [], []
for w in words:
    
    context = [0]*block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix] # crop and append
        
X = torch.tensor(X)
Y = torch.tensor(Y)

In [78]:
X.shape

torch.Size([228146, 3])

In [79]:
g = torch.Generator().manual_seed(2147483647)
output_dim = 2 #acceptable since we only have 27 chars in our model
vocab_size = 27
C = torch.randn((vocab_size,output_dim),generator=g)
W1 = torch.randn((6,100),generator=g)
b1 = torch.randn(100,generator=g)
W2 = torch.randn((100,27),generator=g)
b2 = torch.randn(27,generator=g)
parameters = [C,W1,b1,W2,b2]

In [80]:
sum(p.nelement() for p in parameters)

3481

In [81]:
# forward pass
h = torch.tanh(C[X].view(-1,6)@ W1+b1)
logits = h @ W2 + b2
# counts = logits.exp()
# prob = counts / counts.sum(1,keepdims=True)
# loss = -prob[torch.arange(32),Y].log().mean()
loss2 = F.cross_entropy(logits,Y)
loss2

tensor(19.5052)

In [82]:
for p in parameters:
    p.requires_grad= True

In [85]:
epoch = 100
for _ in range(epoch):
    # forward pass
    emb = C[X] # shape (32,3,2)
    h = torch.tanh(emb.view(-1,6) @ W1+b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits,Y)
    print(loss.item())
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    #update
    for p in parameters:
        p.data += -0.1 * p.grad
print(loss.item())

8.722230911254883
8.521748542785645
8.32922649383545
8.144325256347656
7.966790676116943
7.796450614929199
7.633184909820557
7.476907253265381
7.327520370483398
7.184885025024414
7.048791885375977
6.918952465057373
6.795018196105957
6.676602840423584
6.563317775726318
6.454789161682129
6.350668907165527
6.250642776489258
6.154431343078613
6.061785697937012
5.972482204437256
5.886327743530273
5.803146839141846
5.722784042358398
5.645094394683838
5.569945335388184
5.497213363647461
5.4267802238464355
5.358535289764404
5.292375087738037
5.228203296661377
5.165927886962891
5.105468273162842
5.04674768447876
4.98969841003418
4.934261322021484
4.880379676818848
4.828005790710449
4.777095794677734
4.727609157562256
4.679513454437256
4.632778644561768
4.587378025054932
4.543288230895996
4.500490665435791
4.4589667320251465
4.418700218200684
4.379675388336182
4.3418779373168945
4.305292129516602
4.269898891448975
4.235679626464844
4.202611923217773
4.1706695556640625
4.139823913574219
4.1100440

In [119]:
# Mini-batch GD
epoch = 20000
batch_size = 64
for _ in range(epoch):
    # construct mini-batch
    ixs = torch.randint(0,X.shape[0],(batch_size,))
    mini_batch = X[ixs,:]
    mini_batch_y = Y[ixs]
    
    # forward pass
    emb = C[mini_batch]
    h = torch.tanh(emb.view(-1,6) @ W1+b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, mini_batch_y)
#     print(loss.item())
    
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    #update
    for p in parameters:
        p.data += -0.005 * p.grad

In [120]:
emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1+b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss.item()

2.3829360008239746

In [ ]:
2.09